In [115]:
import torch
import torch.nn as nn 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import torch.nn.functional as F


In [116]:
wine_data = pd.read_csv('https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/master/winequality-red.csv')
wine_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [117]:
X = wine_data.drop('quality', axis = 1)
y = wine_data['quality']

In [118]:
X = X.to_numpy()
y = y.to_numpy()

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42, shuffle = True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

y_train = y_train
y_test = y_test

In [120]:
class model(nn.Module):
    def __init__(self, in_features = 11, h1_output = 8, h2_output = 4, out_features = 10):
        super().__init__()
        self.h1 = nn.Linear(in_features, h1_output)
        self.h2 = nn.Linear(h1_output, h2_output)
        self.out = nn.Linear(h2_output, out_features)
    
    def forward(self, X):
        X = F.relu(self.h1(X))
        X = F.relu(self.h2(X))
        X = F.softmax(self.out(X), dim = 1)

        return X

In [121]:
model = model()

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = .01)

In [122]:
epochs = 500
losses = []

for epoch in range(epochs):
    y_pred = model.forward(X_train)

    loss = loss_fn(y_pred, y_train)

    losses.append(loss.detach().numpy())

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if(epoch%10 == 0):
        print(f'Epoch: {epoch}, loss: {loss.detach().numpy()}')


Epoch: 0, loss: 2.348125696182251
Epoch: 10, loss: 2.2917656898498535
Epoch: 20, loss: 2.2506840229034424
Epoch: 30, loss: 2.1689884662628174
Epoch: 40, loss: 2.031416416168213
Epoch: 50, loss: 2.0167880058288574
Epoch: 60, loss: 1.970512866973877
Epoch: 70, loss: 1.9561073780059814
Epoch: 80, loss: 1.949344515800476
Epoch: 90, loss: 1.9452476501464844
Epoch: 100, loss: 1.941028356552124
Epoch: 110, loss: 1.9356005191802979
Epoch: 120, loss: 1.9308308362960815
Epoch: 130, loss: 1.9256612062454224
Epoch: 140, loss: 1.9206095933914185
Epoch: 150, loss: 1.9155254364013672
Epoch: 160, loss: 1.9097431898117065
Epoch: 170, loss: 1.8990960121154785
Epoch: 180, loss: 1.8975235223770142
Epoch: 190, loss: 1.8875007629394531
Epoch: 200, loss: 1.885252594947815
Epoch: 210, loss: 1.8816171884536743
Epoch: 220, loss: 1.8789758682250977
Epoch: 230, loss: 1.8768796920776367
Epoch: 240, loss: 1.875440001487732
Epoch: 250, loss: 1.8738787174224854
Epoch: 260, loss: 1.8723937273025513
Epoch: 270, loss: 1

In [123]:
with torch.no_grad():
    correct = 0
    y_test_pred = model.forward(X_test)
    for i in range(y_test_pred.shape[0]):
        if(y_test_pred[i].argmax() == y_test[i]):
            correct += 1
        print(y_test_pred[i].argmax(), y_test[i])

print(f'Accuracy: {correct}/{y_test_pred.shape[0]}')

tensor(5) tensor(6)
tensor(5) tensor(5)
tensor(6) tensor(6)
tensor(5) tensor(5)
tensor(6) tensor(6)
tensor(5) tensor(5)
tensor(5) tensor(5)
tensor(5) tensor(5)
tensor(6) tensor(5)
tensor(6) tensor(6)
tensor(6) tensor(7)
tensor(5) tensor(3)
tensor(6) tensor(5)
tensor(5) tensor(5)
tensor(5) tensor(6)
tensor(6) tensor(7)
tensor(5) tensor(5)
tensor(5) tensor(7)
tensor(6) tensor(8)
tensor(5) tensor(5)
tensor(6) tensor(5)
tensor(5) tensor(6)
tensor(6) tensor(5)
tensor(6) tensor(6)
tensor(5) tensor(6)
tensor(5) tensor(6)
tensor(6) tensor(7)
tensor(5) tensor(6)
tensor(5) tensor(5)
tensor(6) tensor(6)
tensor(5) tensor(5)
tensor(6) tensor(5)
tensor(6) tensor(6)
tensor(5) tensor(5)
tensor(5) tensor(6)
tensor(5) tensor(5)
tensor(6) tensor(7)
tensor(6) tensor(5)
tensor(6) tensor(4)
tensor(6) tensor(6)
tensor(5) tensor(5)
tensor(5) tensor(5)
tensor(6) tensor(7)
tensor(5) tensor(5)
tensor(6) tensor(5)
tensor(6) tensor(6)
tensor(6) tensor(7)
tensor(5) tensor(6)
tensor(5) tensor(5)
tensor(6) tensor(6)
